In [1]:
import pandas as pd
import numpy as np

#Libraries to help with data visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# To scale the data using z-score 
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Algorithms to use
from sklearn.linear_model import LogisticRegression

# Metrics to evaluate the model
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve,recall_score
from sklearn import tree

# For tuning the model
from sklearn.model_selection import GridSearchCV

# To ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
survey = pd.read_csv('Surveydata_train.csv')
travel = pd.read_csv('Traveldata_train.csv')

In [4]:
#Dimensions of the survey df
survey.shape

(94379, 17)

In [5]:
#Dimensions of the travel df
travel.shape

(94379, 9)

In [3]:
#Merging the two dataframes, since they have the unique ID column in common
df = travel.merge(survey, how='outer', on='ID', indicator=True)
df

,ID,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Overall_Experience,...,Onboard_Entertainment,Online_Support,Ease_of_Online_Booking,Onboard_Service,Legroom,Baggage_Handling,CheckIn_Service,Cleanliness,Online_Boarding,_merge
0,98800001,Female,Loyal Customer,52.0,NaN,Business,272,0.0,5.0,0,...,Needs Improvement,Acceptable,Needs Improvement,Needs Improvement,Acceptable,Needs Improvement,Good,Needs Improvement,Poor,both
1,98800002,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0,0,...,Poor,Good,Good,Excellent,Needs Improvement,Poor,Needs Improvement,Good,Good,both
2,98800003,Female,Loyal Customer,43.0,Business Travel,Business,1061,77.0,119.0,1,...,Good,Excellent,Excellent,Excellent,Excellent,Excellent,Good,Excellent,Excellent,both
3,98800004,Female,Loyal Customer,44.0,Business Travel,Business,780,13.0,18.0,0,...,Needs Improvement,Acceptable,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Acceptable,both
4,98800005,Female,Loyal Customer,50.0,Business Travel,Business,1981,0.0,0.0,1,...,Good,Excellent,Good,Good,Good,Good,Good,Good,Good,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94374,98894375,Male,Loyal Customer,32.0,Business Travel,Business,1357,83.0,125.0,0,...,Poor,Poor,Poor,Good,Good,Good,Needs Improvement,Good,Poor,both
94375,98894376,Male,Loyal Customer,44.0,Business Travel,Business,592,5.0,11.0,1,...,Excellent,Excellent,Acceptable,Acceptable,Acceptable,Acceptable,Good,Acceptable,Good,both
94376,98894377,Male,NaN,63.0,Business Travel,Business,2794,0.0,0.0,1,...,Excellent,Good,Good,Good,Good,Good,Acceptable,Good,Acceptable,both
94377,98894378,Male,Loyal Customer,16.0,Personal Travel,Eco,2744,0.0,0.0,0,...,Needs Improvement,Good,Good,Acceptable,Good,Good,Good,Excellent,Good,both


In [10]:
#EDA:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94379 entries, 0 to 94378
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   ID                       94379 non-null  int64   
 1   Gender                   94302 non-null  object  
 2   Customer_Type            85428 non-null  object  
 3   Age                      94346 non-null  float64 
 4   Type_Travel              85153 non-null  object  
 5   Travel_Class             94379 non-null  object  
 6   Travel_Distance          94379 non-null  int64   
 7   Departure_Delay_in_Mins  94322 non-null  float64 
 8   Arrival_Delay_in_Mins    94022 non-null  float64 
 9   Overall_Experience       94379 non-null  int64   
 10  Seat_Comfort             94318 non-null  object  
 11  Seat_Class               94379 non-null  object  
 12  Arrival_Time_Convenient  85449 non-null  object  
 13  Catering                 85638 non-null  object  
 14  Platfo

**Numeric Columns: ID, Age, Travel_Distance, Departure_Delay_in_Mins, Arrival_Delay_in_Mins, Overall_Experience.**

**Categorical Columns: Gender, Customer_Type, Type_Travel, Travel_Class, Seat_Comfort, Seat_Class, Arrival_Time_Convenient, Catering, Platform_Location, Onboard_Wifi_Service, Onboard_Entertainment, Online_Support, Ease_of_Online_Booking, Onboard_Service, Legroom, Baggage_Handling, CheckIn_Service, Cleanliness, Online_Boarding.**

In [11]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,94379.0,9.884719e+07,27245.014865,98800001.0,98823595.5,98847190.0,98870784.5,98894379.0
Age,94346.0,3.941965e+01,15.116632,7.0,27.0,40.0,51.0,85.0
Travel_Distance,94379.0,1.978888e+03,1027.961019,50.0,1359.0,1923.0,2538.0,6951.0
Departure_Delay_in_Mins,94322.0,1.464709e+01,38.138781,0.0,0.0,0.0,12.0,1592.0
Arrival_Delay_in_Mins,94022.0,1.500522e+01,38.439409,0.0,0.0,0.0,13.0,1584.0
Overall_Experience,94379.0,5.466576e-01,0.497821,0.0,0.0,1.0,1.0,1.0


In [12]:
df.describe(include='object').T

,count,unique,top,freq
Gender,94302,2,Female,47815
Customer_Type,85428,2,Loyal Customer,69823
Type_Travel,85153,2,Business Travel,58617
Travel_Class,94379,2,Eco,49342
Seat_Comfort,94318,6,Acceptable,21158
Seat_Class,94379,2,Green Car,47435
Arrival_Time_Convenient,85449,6,Good,19574
Catering,85638,6,Acceptable,18468
Platform_Location,94349,6,Manageable,24173
Onboard_Wifi_Service,94349,6,Good,22835


In [4]:
num_cols = df.describe().columns.tolist()
cat_cols = df.describe(include='object').columns.tolist()

In [5]:
#Lets Deal with null/missing values
df.isna().sum().sort_values(ascending=False)

Type_Travel                9226
Customer_Type              8951
Arrival_Time_Convenient    8930
Catering                   8741
Onboard_Service            7601
Arrival_Delay_in_Mins       357
Baggage_Handling            142
Online_Support               91
Legroom                      90
CheckIn_Service              77
Gender                       77
Ease_of_Online_Booking       73
Seat_Comfort                 61
Departure_Delay_in_Mins      57
Age                          33
Platform_Location            30
Onboard_Wifi_Service         30
Onboard_Entertainment        18
Online_Boarding               6
Cleanliness                   6
ID                            0
Seat_Class                    0
Overall_Experience            0
Travel_Distance               0
Travel_Class                  0
_merge                        0
dtype: int64

In [5]:
#Methods which help deal with null values

def numNull(df, col):
    median = df[col].median()
    df[col].fillna(median, inplace=True)

def catNull(df, col):
    mode = df[col].mode().iloc[0]
    df[col].fillna(mode, inplace=True)
    
for col in num_cols:
    numNull(df, col)

for col in cat_cols:
    catNull(df, col)

In [8]:
df.isna().sum().sort_values(ascending=False)

ID                         0
Gender                     0
Online_Boarding            0
Cleanliness                0
CheckIn_Service            0
Baggage_Handling           0
Legroom                    0
Onboard_Service            0
Ease_of_Online_Booking     0
Online_Support             0
Onboard_Entertainment      0
Onboard_Wifi_Service       0
Platform_Location          0
Catering                   0
Arrival_Time_Convenient    0
Seat_Class                 0
Seat_Comfort               0
Overall_Experience         0
Arrival_Delay_in_Mins      0
Departure_Delay_in_Mins    0
Travel_Distance            0
Travel_Class               0
Type_Travel                0
Age                        0
Customer_Type              0
_merge                     0
dtype: int64

In [6]:
# Creating list of dummy columns
to_get_dummies_for = ['ID', '_merge']

# Creating dummy variables
df = pd.get_dummies(data = df, columns = to_get_dummies_for, drop_first = True)

In [80]:
df.head()

,Gender,Customer_Type,Age,Type_Travel,Travel_Class,Travel_Distance,Departure_Delay_in_Mins,Arrival_Delay_in_Mins,Overall_Experience,Seat_Comfort,...,ID_98894372,ID_98894373,ID_98894374,ID_98894375,ID_98894376,ID_98894377,ID_98894378,ID_98894379,_merge_right_only,_merge_both
0,Female,Loyal Customer,52.0,Business Travel,Business,272,0.0,5.0,0,Needs Improvement,...,0,0,0,0,0,0,0,0,0,1
1,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0,0,Poor,...,0,0,0,0,0,0,0,0,0,1
2,Female,Loyal Customer,43.0,Business Travel,Business,1061,77.0,119.0,1,Needs Improvement,...,0,0,0,0,0,0,0,0,0,1
3,Female,Loyal Customer,44.0,Business Travel,Business,780,13.0,18.0,0,Acceptable,...,0,0,0,0,0,0,0,0,0,1
4,Female,Loyal Customer,50.0,Business Travel,Business,1981,0.0,0.0,1,Acceptable,...,0,0,0,0,0,0,0,0,0,1


In [8]:
# Separating target variable and other variables

Y= df.Overall_Experience
X= df.drop(columns = ['Overall_Experience'])

In [10]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1, stratify = Y)

In [ ]:
# Scaling the data
sc=StandardScaler()

# Fit_transform on train data
X_train_scaled=sc.fit_transform(X_train)
X_train_scaled=pd.DataFrame(X_train_scaled, columns=X.columns)

In [ ]:
# Transform on test data
X_test_scaled=sc.transform(X_test)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=X.columns)

In [ ]:
#Scaling Without Splitting the data into Validation data (ONLY run if not splitting)
scaler = StandardScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
def metrics_score(actual, predicted):
    print(classification_report(actual, predicted))

    cm = confusion_matrix(actual, predicted)
    plt.figure(figsize=(8,5))
    
    sns.heatmap(cm, annot=True,  fmt='.2f', xticklabels=['Not Attrite', 'Attrite'], yticklabels=['Not Attrite', 'Attrite'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
# Fitting logistic regression model

lg=LogisticRegression()
lg.fit(X_train,y_train)

In [ ]:
# Checking the performance on the training data
y_pred_train = lg.predict(X_train_scaled)

metrics_score(y_train, y_pred_train)